In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [11]:
class GMM(object):
    def __init__(self, X, k = 3):
        X = np.asarray(X)
        self.m, self.n = X.shape
        self.data = X.copy()
        self.k = k
        
    def ini(self):
        self.mean_arr = np.asmatrix(np.zeros((self.k, self.n)))
        self.sigma_arr = np.array([np.asmatrix(np.identity(self.n)) for i in range(self.k)])
        self.phi = np.ones(self.k)/self.k
        self.w = np.asmatrix(np.empty((self.m, self.k)), dtype = float)
        
    def fit(self, tol = 1e-4):
        self.ini()
        num_iter = 0
        log_l = 1
        pre_log_l = 0
        while (log_l - pre_log_l > tol):
            pre_log_l = self.loglikelihood()
            self.fit_()
            num_iter += 1
            log_l = self.loglikelihood()
            print("iteration %d: loglikelihood is %.6f"%(num_iter, log_l))
        print("terminate at %d-th iteration: log-likelihood is %6.f"%(num_iter, log_l))
        
    def loglikelihood(self):
        log_l = 0
        for i in range(self.m):
            tmp = 0
            for j in range(self.k):
                tmp += multivariate_normal.pdf(self.data[i, :], 
                                                        self.mean_arr[j, :].A1, 
                                                        self.sigma_arr[j, :]) *\
                       self.phi[j]
            log_l += np.log(tmp)
        return log_l
    
    def fit_(self):
        self.e_step()
        self.m_step()
        
    def e_step(self):
        for i in range(self.m):
            den = 0
            for j in range(self.k):
                num = multivariate_normal.pdf(self.data[i, :], 
                                                       self.mean_arr[j].A1, 
                                                       self.sigma_arr[j]) *\
                      self.phi[j]
                den += num
                self.w[i, j] = num
            self.w[i, :] /= den
            assert self.w[i, :].sum() - 1 < 1e-4
            
    def m_step(self):
        for j in range(self.k):
            const = self.w[:, j].sum()
            self.phi[j] = 1/self.m * const
            mu_j = np.zeros(self.n)
            sigma_j = np.zeros((self.n, self.n))
            for i in range(self.m):
                mu_j += (self.data[i,:] * self.w[i,j])
                sigma_j += self.w[i,j] * ((self.data[i,:] - self.mean_arr[j, :]).T * (self.data[i, :] - self.mean_arr[j, :]))
            self.mean_arr[j] = mu_j / const
            self.sigma_arr[j] = sigma_j / const
            

In [12]:
data = np.loadtxt("data.dat").transpose()
data.shape


(1990, 784)

In [10]:
gmm = GMM(data)
gmm.fit()

/usr/local/Cellar/ipython/7.2.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in log


KeyboardInterrupt: 